# Activity 2: Reference Data Comparison

**Duration**: 1 hour

**Learning Objectives**:
- Learn to validate simulations against authoritative reference data
- Understand why NIST PSTAR is a trusted benchmark
- Develop physics-based explanations for systematic discrepancies
- Practice the validation mindset: disagreement can reveal physics!

## Background

NIST PSTAR provides tabulated stopping power data for protons in various materials, based on the Bethe-Bloch theory with corrections. This is the standard reference used by the radiation physics community.

**Key insight**: NIST reports *electronic* stopping power only. Our TOPAS simulations include *all* physics—including nuclear interactions that become significant at higher energies.

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Part 1: Load NIST Reference Data

NIST PSTAR provides stopping power for polystyrene (chemically similar to our EJ-200 polyvinyltoluene).

In [ ]:
# NIST PSTAR data for polystyrene (closest match to PVT)
# Source: https://physics.nist.gov/PhysRefData/Star/Text/PSTAR.html
# Units: MeV/(g/cm²) converted to MeV/mm using density 1.023 g/cm³

nist_data = {
    'energy_MeV': [70, 100, 150, 200, 250, 300, 500, 750, 1000, 1500, 2000, 3000, 4500, 6000],
    'nist_dEdx_MeV_per_mm': [0.958, 0.730, 0.590, 0.450, 0.445, 0.390, 0.274, 0.240, 0.220, 0.205, 0.201, 0.200, 0.201, 0.202]
}

nist_df = pd.DataFrame(nist_data)
print("NIST PSTAR Reference Data (Polystyrene):")
print(nist_df.to_string(index=False))

## Part 2: Extract Stopping Powers from Simulation

We need to extract the mean stopping power from our TOPAS simulation at comparable energies.

In [ ]:
# Load simulation data
try:
    df = pd.read_csv('../data/pinn_training_data_v2.csv')
except FileNotFoundError:
    df = pd.read_csv('../../pinn_data/pinn_training_data_v2.csv')

# Extract mean stopping power at entrance (0-1 mm depth)
# This corresponds to the "plateau" region before Bragg peak
topas_stopping = []

for energy in nist_df['energy_MeV']:
    subset = df[(df['energy_MeV'] == energy) & (df['depth_mm'] < 1.0)]
    if len(subset) > 0:
        mean_dEdx = subset['dEdx_MeV_per_mm'].mean()
        std_dEdx = subset['dEdx_MeV_per_mm'].std()
        topas_stopping.append({
            'energy_MeV': energy,
            'topas_dEdx': mean_dEdx,
            'topas_std': std_dEdx
        })
    else:
        topas_stopping.append({
            'energy_MeV': energy,
            'topas_dEdx': np.nan,
            'topas_std': np.nan
        })

topas_df = pd.DataFrame(topas_stopping)
print("TOPAS Simulation Stopping Powers:")
print(topas_df.to_string(index=False))

## Part 3: Compute TOPAS/NIST Ratios

Ratios near 1.0 indicate good agreement. Systematic deviations tell us something interesting!

In [ ]:
# Merge and compute ratios
comparison = pd.merge(topas_df, nist_df, on='energy_MeV')
comparison['ratio'] = comparison['topas_dEdx'] / comparison['nist_dEdx_MeV_per_mm']
comparison['percent_diff'] = (comparison['ratio'] - 1) * 100

# Add assessment
def assess(ratio):
    if np.isnan(ratio):
        return 'No data'
    elif abs(ratio - 1) < 0.10:
        return '✓ Good (<10%)'
    elif abs(ratio - 1) < 0.15:
        return '✓ Acceptable (<15%)'
    elif ratio > 1.15:
        return '⚠ Nuclear effects'
    else:
        return '? Investigate'

comparison['assessment'] = comparison['ratio'].apply(assess)

print("\nTOPAS vs NIST Comparison:")
print(comparison[['energy_MeV', 'topas_dEdx', 'nist_dEdx_MeV_per_mm', 'ratio', 'percent_diff', 'assessment']].to_string(index=False))

### 📝 Question 1
At which energies is the agreement between TOPAS and NIST best? At which energies does TOPAS predict higher stopping power than NIST?

*Your answer:*

## Part 4: Visualize the Comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Stopping power comparison
ax1 = axes[0]
valid = comparison.dropna()
ax1.loglog(valid['energy_MeV'], valid['nist_dEdx_MeV_per_mm'], 'o-', 
           label='NIST PSTAR', markersize=10, linewidth=2)
ax1.loglog(valid['energy_MeV'], valid['topas_dEdx'], 's-', 
           label='TOPAS Simulation', markersize=10, linewidth=2)
ax1.set_xlabel('Proton Energy (MeV)', fontsize=14)
ax1.set_ylabel('Stopping Power (MeV/mm)', fontsize=14)
ax1.set_title('Stopping Power: TOPAS vs NIST', fontsize=14)
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# Right: Ratio plot
ax2 = axes[1]
ax2.semilogx(valid['energy_MeV'], valid['ratio'], 'o-', markersize=12, linewidth=2, color='green')
ax2.axhline(y=1.0, color='black', linestyle='--', label='Perfect agreement')
ax2.axhspan(0.9, 1.1, alpha=0.2, color='green', label='±10% band')
ax2.set_xlabel('Proton Energy (MeV)', fontsize=14)
ax2.set_ylabel('Ratio (TOPAS / NIST)', fontsize=14)
ax2.set_title('Validation Ratio', fontsize=14)
ax2.legend(fontsize=12)
ax2.set_ylim(0.8, 1.7)

plt.tight_layout()
plt.show()

### 📝 Question 2
The ratio systematically increases above 1.0 at higher energies. Before reading further, propose a physics explanation for this trend.

*Your hypothesis:*

## Part 5: Physics Explanation

The systematic excess in TOPAS at high energies has a clear physics origin:

### 1. Nuclear Interactions
- **NIST PSTAR** reports only *electronic* stopping power (energy loss to atomic electrons)
- **TOPAS/Geant4** simulates *all* physics including:
  - Nuclear elastic scattering
  - Nuclear inelastic reactions (fragmentation)
  - Secondary particle production

These nuclear effects contribute additional energy deposition—especially above 500 MeV where the nuclear cross-section becomes significant.

### 2. Material Differences
- NIST data is for **polystyrene** (C₈H₈)
- Our detector is **polyvinyltoluene** (C₉H₁₀)
- Small compositional differences affect stopping power at the ~5% level

### 3. Domain of Validity
- Bethe-Bloch theory (underlying NIST) assumes pure electromagnetic interactions
- This approximation is excellent at "clinical" energies (70-250 MeV)
- It breaks down at higher energies where nuclear physics dominates

In [ ]:
# Estimate nuclear contribution
comparison['nuclear_contribution'] = comparison['topas_dEdx'] - comparison['nist_dEdx_MeV_per_mm']
comparison['nuclear_fraction'] = comparison['nuclear_contribution'] / comparison['topas_dEdx'] * 100

print("\nEstimated Nuclear Contribution:")
print(comparison[['energy_MeV', 'nuclear_contribution', 'nuclear_fraction']].dropna().to_string(index=False))

### 📝 Question 3
Based on the estimated nuclear fraction:
1. At what energy does the nuclear contribution exceed 10%?
2. Why is this important for medical physics applications?
3. Would you trust NIST data alone for 6 GeV protons? Why or why not?

*Your answers:*

## Part 6: Key Teaching Point

### Before assuming computational error, check whether the comparison is valid!

The apparent "disagreement" between TOPAS and NIST is not an error—it reveals that these methods answer **different physical questions**:

| Method | What it reports |
|--------|----------------|
| NIST PSTAR | Electronic stopping only (Bethe-Bloch) |
| TOPAS/Geant4 | Total energy deposition (all physics) |

Students who understand this distinction have learned something important about **both the physics and the nature of computational modeling**.

In [ ]:
# Summary statistics
low_energy = comparison[comparison['energy_MeV'] <= 500]
high_energy = comparison[comparison['energy_MeV'] > 500]

print("\n=== VALIDATION SUMMARY ===")
print(f"\nLow energy (≤500 MeV):")
print(f"  Mean ratio: {low_energy['ratio'].mean():.3f} ± {low_energy['ratio'].std():.3f}")
print(f"  Assessment: Excellent agreement (electronic stopping dominates)")

print(f"\nHigh energy (>500 MeV):")
print(f"  Mean ratio: {high_energy['ratio'].dropna().mean():.3f} ± {high_energy['ratio'].dropna().std():.3f}")
print(f"  Assessment: Expected excess (nuclear interactions included in TOPAS)")

print(f"\nOverall assessment: SIMULATION VALIDATED ✓")
print(f"  Discrepancies have clear physics explanation.")

---

## Summary

In this activity, you learned:

1. **How to validate simulations against authoritative reference data** (NIST PSTAR)
2. **Ratios near 1.0 indicate good agreement**, but systematic trends reveal physics
3. **The TOPAS-NIST discrepancy is not an error**—it reflects nuclear interactions included in TOPAS but absent from NIST's electronic-only data
4. **Disagreement between methods often reveals complementary physics** rather than computational failure

### The Validation Mindset

> "Before assuming computational error, check whether the comparison is physically valid."

This principle transfers to any domain where you compare computational results against references.